<a href="https://colab.research.google.com/github/fabiobaroliveira/gerador_base_de_dados/blob/main/locadora_de_ve%C3%ADculos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Criando base de dados de uma locadora de veículos

---
- Clientes
- Locações
- Frota de veículos


In [1]:
#Necessário instalar a biblioteca faker
! pip install faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 37.1 MB/s eta 0:00:00


In [24]:
import pandas as pd
from faker import Faker
import random
from datetime import datetime, timedelta

# Inicializa o Faker
fake = Faker('pt_BR')

# Configurações
NUM_CLIENTES = 3650  # Número de clientes
NUM_VEICULOS = 50   # Número de veículos
NUM_LOCACOES = 20000 # Número de locações

# Lista de dicionários com modelos e suas categorias
modelos_categorias = [
    {"marca": "Fiat", "modelo": "Mobi", "categoria": "Econômico"},
    {"marca": "Fiat", "modelo": "Argo", "categoria": "Econômico"},
    {"marca": "Fiat", "modelo": "Cronos", "categoria": "Intermediário"},
    {"marca": "Fiat", "modelo": "Pulse", "categoria": "SUV"},
    {"marca": "Fiat", "modelo": "Toro", "categoria": "Pickup"},
    {"marca": "Volkswagen", "modelo": "Gol", "categoria": "Econômico"},
    {"marca": "Volkswagen", "modelo": "Polo", "categoria": "Intermediário"},
    {"marca": "Volkswagen", "modelo": "Virtus", "categoria": "Intermediário"},
    {"marca": "Volkswagen", "modelo": "T-Cross", "categoria": "SUV"},
    {"marca": "Chevrolet", "modelo": "Onix", "categoria": "Econômico"},
    {"marca": "Chevrolet", "modelo": "Cruze", "categoria": "Intermediário"},
    {"marca": "Chevrolet", "modelo": "Tracker", "categoria": "SUV"},
    {"marca": "Chevrolet", "modelo": "S10", "categoria": "Pickup"},
    {"marca": "Hyundai", "modelo": "HB20", "categoria": "Econômico"},
    {"marca": "Hyundai", "modelo": "Creta", "categoria": "SUV"},
    {"marca": "Hyundai", "modelo": "Tucson", "categoria": "SUV"},
    {"marca": "Toyota", "modelo": "Corolla", "categoria": "Intermediário"},
    {"marca": "Toyota", "modelo": "Hilux", "categoria": "Pickup"},
    {"marca": "Toyota", "modelo": "SW4", "categoria": "SUV"},
    {"marca": "Ford", "modelo": "Ka", "categoria": "Econômico"},
    {"marca": "Ford", "modelo": "EcoSport", "categoria": "SUV"},
    {"marca": "Ford", "modelo": "Ranger", "categoria": "Pickup"}
]

# Listas de opções
cores = ["Preto", "Branco", "Prata", "Vermelho", "Azul", "Cinza"]
status_locacao = ["Ativa", "Finalizada", "Cancelada"]

# Função para gerar datas aleatórias
def gerar_data_aleatoria(inicio, fim):
    return inicio + timedelta(days=random.randint(0, (fim - inicio).days))



# Gerar clientes

In [25]:
clientes = []
for i in range(1, NUM_CLIENTES + 1):
    sexo = random.choice(["Masculino", "Feminino"])
    nome = fake.first_name_male() if sexo == "Masculino" else fake.first_name_female()
    clientes.append({
        "id_cliente": i,
        "nome": f"{nome} {fake.last_name()}",
        "cpf": fake.cpf(),
        "email": fake.email(),
        "telefone": fake.phone_number(),
        "data_nascimento": fake.date_of_birth(minimum_age=18, maximum_age=70),
        "sexo": sexo,
        "cidade": fake.city(),
        "estado": fake.state_abbr()
    })

df_clientes = pd.DataFrame(clientes)

# Gerar veículos

In [26]:
# Gerar veículos
veiculos = []
for i in range(1, NUM_VEICULOS + 1):
    veiculo = random.choice(modelos_categorias)  # Escolhe um veículo aleatório da lista
    ano = random.randint(2015, 2023)
    placa = f"{random.choice(['ABC', 'DEF', 'GHI'])}{random.randint(1000, 9999)}"
    veiculos.append({
        "id_veiculo": i,
        "marca": veiculo["marca"],
        "modelo": veiculo["modelo"],
        "ano": ano,
        "placa": placa,
        "categoria": veiculo["categoria"],
        "cor": random.choice(cores),
        "quilometragem": random.randint(0, 100000),
        "disponivel": random.choice([True, False])
    })

df_veiculos = pd.DataFrame(veiculos)

#Gerar locações

In [27]:
# Gerar locações
locacoes = []
data_inicio = datetime(2020, 1, 1)
data_fim = datetime(2024, 12, 31)

for i in range(1, NUM_LOCACOES + 1):
    id_cliente = random.randint(1, NUM_CLIENTES)
    id_veiculo = random.randint(1, NUM_VEICULOS)
    data_locacao = gerar_data_aleatoria(data_inicio, data_fim)
    data_devolucao = data_locacao + timedelta(days=random.randint(1, 30))
    valor_diaria = round(random.uniform(100, 500), 2)
    dias_locacao = (data_devolucao - data_locacao).days
    valor_total = round(valor_diaria * dias_locacao, 2)
    status = random.choice(status_locacao)

    locacoes.append({
        "id_locacao": i,
        "id_cliente": id_cliente,
        "id_veiculo": id_veiculo,
        "data_locacao": data_locacao.strftime('%Y-%m-%d'),
        "data_devolucao": data_devolucao.strftime('%Y-%m-%d'),
        "valor_diaria": valor_diaria,
        "dias_locacao": dias_locacao,
        "valor_total": valor_total,
        "status": status
    })


df_locacoes = pd.DataFrame(locacoes)
#substituindo . dos valores e ajustando ao padrão brasileiro
df_locacoes['valor_diaria'] = df_locacoes['valor_diaria'].astype(str).str.replace('.', ',')
df_locacoes['valor_total'] = df_locacoes['valor_total'].astype(str).str.replace('.', ',')


# Exibir DataFrames

In [28]:
print("Clientes:")
df_clientes.head()

Clientes:


,id_cliente,nome,cpf,email,telefone,data_nascimento,sexo,cidade,estado
0,1,Asafe Vasconcelos,709.452.138-32,vnovais@example.net,71 3267-3354,2000-10-30,Masculino,Viana de Goiás,MG
1,2,Amanda das Neves,674.320.859-00,maitemoreira@example.net,+55 84 7135 0884,1964-07-04,Feminino,Rios da Prata,AC
2,3,Nicolas Pimenta,364.517.802-35,isaque10@example.org,+55 84 0867-9384,1986-11-07,Masculino,Cassiano,RN
3,4,Ana Beatriz Silveira,439.761.025-80,joaquim85@example.net,(051) 9723 0983,1973-02-16,Feminino,Pereira,MA
4,5,Enrico Moraes,038.645.291-15,yanbrito@example.com,51 0193 8565,1979-11-14,Masculino,Cardoso Grande,AL


In [29]:
print("\nVeículos:")
df_veiculos.head()


Veículos:


,id_veiculo,marca,modelo,ano,placa,categoria,cor,quilometragem,disponivel
0,1,Ford,Ranger,2016,GHI4071,Pickup,Vermelho,70888,True
1,2,Chevrolet,Tracker,2020,GHI4363,SUV,Vermelho,284,True
2,3,Ford,EcoSport,2021,GHI1950,SUV,Branco,28389,False
3,4,Chevrolet,Tracker,2016,GHI6273,SUV,Prata,44892,False
4,5,Volkswagen,Virtus,2016,DEF9014,Intermediário,Preto,87728,False


In [30]:
print("\nLocações:")
df_locacoes.head()


Locações:


,id_locacao,id_cliente,id_veiculo,data_locacao,data_devolucao,valor_diaria,dias_locacao,valor_total,status
0,1,1140,34,2022-11-14,2022-12-07,"449,65",23,"10341,95",Cancelada
1,2,2464,43,2022-11-10,2022-12-03,"260,99",23,"6002,77",Cancelada
2,3,2857,26,2023-04-08,2023-04-17,"189,66",9,"1706,94",Finalizada
3,4,323,8,2023-02-18,2023-02-21,"165,13",3,"495,39",Cancelada
4,5,351,1,2020-09-29,2020-10-03,"454,43",4,"1817,72",Finalizada


#Criando os arquivos .csv

In [23]:
df_clientes.to_csv('clientes.csv', index=False, sep=';')
df_veiculos.to_csv('veiculos.csv', index=False, sep=';')
df_locacoes.to_csv('locacoes.csv', index=False, sep=';')
